In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
OPENAI_API_KEY = config['OPENAI']['OPENAI_API_KEY']
YOUTUBE_KEY = config['YOUTUBE']['YOUTUBE_API_KEY']
NAVER_CLIENT_ID = config['NAVER']['NAVER_CLIENT_ID']
NAVER_CLIENT_SECRET = config['NAVER']['NAVER_CLIENT_SECRET']
GOOGLE_SEARCH_KEY = config['GOOGLE']['GOOGLE_API_KEY']
CSE_ID = config['GOOGLE']['CSE_ID']
SERPAPI_API_KEY = config['SERPAPI']['SERPAPI_API_KEY']


os.environ.update({'OPENAI_API_KEY': OPENAI_API_KEY})
os.environ.update({'YOUTUBE_KEY': YOUTUBE_KEY})
os.environ.update({'NAVER_CLIENT_ID': NAVER_CLIENT_ID})
os.environ.update({'NAVER_CLIENT_SECRET': NAVER_CLIENT_SECRET})
os.environ.update({'GOOGLE_SEARCH_KEY': GOOGLE_SEARCH_KEY})
os.environ.update({'CSE_ID': CSE_ID})
os.environ.update({'SERPAPI_API_KEY': SERPAPI_API_KEY})

### Get tools

In [3]:
from models.llm.chain import GraphChain, DraftChunkChain

In [4]:
graph_template_prompt_path = '../../openai_skt/models/templates/graph_prompt_template.txt'
with open(graph_template_prompt_path, 'r') as f:
    graph_template = f.read()
graph_chain = GraphChain(graph_template=graph_template, input_variables=["graph_to_draw"])

In [5]:
draft_chunk_template_prompt_path = '../../openai_skt/models/templates/draft_chunk_prompt_template.txt'
with open(draft_chunk_template_prompt_path, 'r') as f:
    draft_chunk_template = f.read()
draft_chunk_chain = DraftChunkChain(draft_chunk_template=draft_chunk_template, input_variables=["draft", "query"])

In [5]:
from tools import DatabaseTool, SearchTool, SearchByURLTool, TimeTool, DraftChunkTool, GraphTool
from models.llm.chain import SummaryChunkChain

In [6]:
summary_chunk_template = '''
I want you to answer the question from the given documentation. 
There may be a lot of information in a given document that is not relevant to your question, but you should try to find as much relevant information as possible.
Be as diligent as possible in returning your answer based on your knowledge and documentation.
If there is no answer in the document, return "No useful information".

Document: {document}
Question: {question}
Answer:
'''

In [7]:
summary_chunk_chain = SummaryChunkChain(summary_chunk_template=summary_chunk_template, input_variables=["question", "document"], verbose=True)
search_by_url_tool = SearchByURLTool()

In [24]:
from langchain.agents import Tool
from langchain.utilities import SerpAPIWrapper

# You can create the tool to pass to an agent
search = SerpAPIWrapper(params={
        "engine": "google",
        "google_domain": "google.com",
        "gl": "kr",
        "hl": "ko",
        })
search_tool = Tool(
    name="search_tool",
    description="A tool to search on internet.",
    func=search.run,
)

In [8]:
database_tool = DatabaseTool(summary_chunk_chain=summary_chunk_chain)
# draft_chunk_tool = DraftChunkTool(draft_chunk_chain=draft_chunk_chain)
graph_tool = GraphTool(graph_chain=graph_chain)
search_tool = SearchTool(summary_chunk_chain=summary_chunk_chain, search_by_url_tool=search_by_url_tool)
time_tool = TimeTool()

In [9]:
tools = [database_tool, graph_tool, search_tool]

### Set Agent

In [10]:
from models.llm.draft_edit_agent import DraftEditAgent

In [11]:
draft_edit_agent = DraftEditAgent(model='gpt-4', tools=tools, verbose=True, draft_edit_prompt_path='../../openai_skt/models/templates/draft_edit_prompt_template.txt')

In [13]:
from database import DataBase

In [12]:
from database import CustomEmbedChain
embed_chain = CustomEmbedChain()

In [13]:
# database = DataBase.load(database_path='./user/test_2/database.json', embed_chain=embed_chain)
import pickle
with open("database_save_path.pkl", "rb") as f:
    database = pickle.load(f)
database.set_embed_chain(embed_chain)


In [14]:
with open('./user/test_2/draft_0.md', 'r') as f:
    draft = f.read()

In [15]:
print(draft)

# 서론

디지털 자산은 현대 사회에서 점차적으로 중요성을 갖추고 있는 개념입니다. 디지털 자산은 전자적인 형태로 존재하며, 재산적 가치를 가지고 있으며 사용할 권리가 있는 무형자산입니다. 이러한 디지털 자산은 기존의 재산 개념을 넘어서며, 새로운 경제 시스템과 금융 시장을 형성하고 있습니다.

비트코인은 디지털 자산 중에서도 가장 잘 알려진 암호화폐입니다. 2009년에 등장한 비트코인은 블록체인 기술을 기반으로 작동하며, 분산원장을 통해 거래 기록을 관리합니다. 비트코인은 중앙 기관이나 정부의 감독을 받지 않고 운영되며, 익명성과 보안성을 갖추고 있습니다.

비트코인은 전 세계적으로 거래되고 있으며, 전자상거래와 결제 시스템에서 널리 활용되고 있습니다. 또한 글로벌 경제와 금융 시스템에도 영향을 미치고 있으며, 새로운 경제 모델과 기회를 창출하고 있습니다.

하지만 비트코인은 가치의 변동성과 부정적인 영향을 가지고 있습니다. 이러한 문제점들은 국가별로 다양한 규제와 법적인 쟁점을 야기하고 있습니다. 따라서 비트코인의 발전과 활용에는 법적인 쟁점을 해결하고 금융 기관과의 협력이 필요합니다.

이번 글에서는 디지털 자산의 개념과 중요성에 대해 알아보고, 비트코인의 등장과 역할에 대해 살펴보겠습니다. 이를 통해 디지털 자산과 비트코인에 대한 이해를 높이고, 미래의 발전 가능성을 탐구해보겠습니다.
## 비트코인의 작동 원리

### 블록체인 기술과 분산원장

비트코인은 블록체인 기술을 기반으로 작동합니다. 블록체인은 거래 내역을 분산 저장하며, 이를 위해 네트워크에 참여하는 다수의 노드들이 거래 내역을 확인하고 검증하는 과정을 거칩니다. 이 과정을 통해 거래 내역의 위변조를 방지하고, 신뢰성과 보안성을 확보할 수 있습니다.

블록체인은 블록이라는 단위로 거래 내역을 저장합니다. 각 블록은 이전 블록과 연결되어 있으며, 이를 체인 형태로 구성합니다. 따라서 블록체인은 이전의 거래 내역을 모두 포함하고 있으며, 변경이 발생하면 모든 노드들이 이를 확인하고 동기화합니다.

In [16]:
part_draft = """
# 서론

디지털 자산은 현대 사회에서 점차적으로 중요성을 갖추고 있는 개념입니다. 디지털 자산은 전자적인 형태로 존재하며, 재산적 가치를 가지고 있으며 사용할 권리가 있는 무형자산입니다. 이러한 디지털 자산은 기존의 재산 개념을 넘어서며, 새로운 경제 시스템과 금융 시장을 형성하고 있습니다.

비트코인은 디지털 자산 중에서도 가장 잘 알려진 암호화폐입니다. 2009년에 등장한 비트코인은 블록체인 기술을 기반으로 작동하며, 분산원장을 통해 거래 기록을 관리합니다. 비트코인은 중앙 기관이나 정부의 감독을 받지 않고 운영되며, 익명성과 보안성을 갖추고 있습니다.

비트코인은 전 세계적으로 거래되고 있으며, 전자상거래와 결제 시스템에서 널리 활용되고 있습니다. 또한 글로벌 경제와 금융 시스템에도 영향을 미치고 있으며, 새로운 경제 모델과 기회를 창출하고 있습니다.

하지만 비트코인은 가치의 변동성과 부정적인 영향을 가지고 있습니다. 이러한 문제점들은 국가별로 다양한 규제와 법적인 쟁점을 야기하고 있습니다. 따라서 비트코인의 발전과 활용에는 법적인 쟁점을 해결하고 금융 기관과의 협력이 필요합니다.

이번 글에서는 디지털 자산의 개념과 중요성에 대해 알아보고, 비트코인의 등장과 역할에 대해 살펴보겠습니다.
"""

In [17]:
database_tool = DatabaseTool(summary_chunk_chain=summary_chunk_chain)
database_tool.set_database(database=database)
tools = [database_tool, graph_tool, search_tool]

In [19]:
draft_edit_agent.run(tools=tools, draft=part_draft, query="상단에 2021년 비트코인 가격 차트를 넣어줘")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
You should modify the draft according to the user's requirements. Use a variety of tools to answer the questions.
You need to find the material first, then use it to answer.

You have access to the following tools:

database: A tool to get data with from database. The input consists of a 'query' and a 'question', where 'query' is the search query and 'question' is the information you want to get. For example, {'query': 'bitcoin price history', 'question': 'what was the price of bitcoin in 2022?'} would be the input if you want to know the price of bitcoin in 2022. Input must contain both a query and a question.
graph_tool: 
A tool to draw a graph. It return image path of the graph. This tool simply converts the data you give it into a graph, so you'll need to enter a table in markdown form with your data and the features of the graph you want to draw. The example of input is as follows:


'# 서론\n\n디지털 자산은 현대 사회에서 점차적으로 중요성을 갖추고 있는 개념입니다. 디지털 자산은 전자적인 형태로 존재하며, 재산적 가치를 가지고 있으며 사용할 권리가 있는 무형자산입니다. 이러한 디지털 자산은 기존의 재산 개념을 넘어서며, 새로운 경제 시스템과 금융 시장을 형성하고 있습니다.\n\n비트코인은 디지털 자산 중에서도 가장 잘 알려진 암호화폐입니다. 2009년에 등장한 비트코인은 블록체인 기술을 기반으로 작동하며, 분산원장을 통해 거래 기록을 관리합니다. 비트코인은 중앙 기관이나 정부의 감독을 받지 않고 운영되며, 익명성과 보안성을 갖추고 있습니다.\n\n비트코인은 전 세계적으로 거래되고 있으며, 전자상거래와 결제 시스템에서 널리 활용되고 있습니다. 또한 글로벌 경제와 금융 시스템에도 영향을 미치고 있으며, 새로운 경제 모델과 기회를 창출하고 있습니다.\n\n하지만 비트코인은 가치의 변동성과 부정적인 영향을 가지고 있습니다. 이러한 문제점들은 국가별로 다양한 규제와 법적인 쟁점을 야기하고 있습니다. 따라서 비트코인의 발전과 활용에는 법적인 쟁점을 해결하고 금융 기관과의 협력이 필요합니다.\n\n이번 글에서는 디지털 자산의 개념과 중요성에 대해 알아보고, 비트코인의 등장과 역할에 대해 살펴보겠습니다.\n\n2021년 비트코인 가격 차트:\n\n![Bitcoin Price Chart 2021](https://example.com/bitcoin_price_chart_2021.png)'